<a href="https://colab.research.google.com/github/Kim-JungHyun01/AIStudy24/blob/master/%ED%85%8C%EC%8A%A4%ED%8A%B8/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증테스트

- 일반적인 알고리즘 : 훈련세트 & 테스트세트 분류
- 테스트세트로 평가를 하면 테스트세트에 맞는 모델이 만들어짐

    => 과소,과대적합만 안될려고 하기에 <검증>이 필요
- 테스트 세트에 일반화 성능을 올바르게 예측하기 위해서는 가능한 테스트세트를 사용하면 X
    
    => 모델을 만들고 마지막에 딱 한번 사용하는 것이 좋음
- 결정트리의 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝진행
<br/><br/>
- 테스트 세트를 사용하지 않으려면 모델이 과소, 과대적합인지 판단이 어려움
- 테스트 세트를 사용하지 않고 측정하는 방법 : 훈련세트를 또 나누는 것
    - 훈련세트 : 검증세트 : 테스트세트 = 60 : 20 : 20
    - 진행
        1. 테스트하고 싶은 매개변수를 변경(가장 좋은 모델이 나올 수 있도록 하는 것)
        2. 매개변수를 사용해 훈련세트, 검즘세트를 합침
        3. 전체 훈련 데이터에서 모델을 다시 훈련함(훈련 : 훈련세트, 평가 : 검증세트)
        4. 마지막 테스테에서 최종 점수 평가(테스트세트)

# 데이터 input, target
- train : sub(훈련용), val(검증용)
- test : 테스트용
<br/><br/>
- total(100%) => train : test = 80% : 20%
- train(80%) => sub: val = 60% : 20%

In [2]:
import pandas as pd

wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

#tarin, test = 80 : 20 => test_size = 0.2
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

#sub : 검증(val) = 60 : 20
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

print("train : ", train_input.shape, " test : ", test_input.shape)
print("sub : ",sub_input.shape,"val : ",val_input.shape)

train :  (5197, 3)  test :  (1300, 3)
sub :  (4157, 3) val :  (1040, 3)


# 결정트리 훈련(sub)

In [4]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print("sub : ", dt.score(sub_input, sub_target))
print("val : ", dt.score(val_input, val_target))
# 결론 : tarin(sub)(0.997) : train점수 100%에 가까움(과대적합)
# 매개변수 조정필요

sub :  0.9971133028626413
val :  0.864423076923077


# 교차검증(cross_validate)
- 검증세트생성으로 인해 훈련세트의 양이 줄어듬(많은 데이터를 훈련하면 좋은 결과가 나올 가능성이 높음)
- 교차검증 진행
    - 훈련, 검증세트를 나누어 평가하는 방식을 여러번 반복하는 것
    - ex) 3-폴드, 5-폴드(기본), 10-폴드
- 결과
    - (k,v)의 딕셔너리형태로 반환
        1. fit_time(훈련시간)
        2. score_time(검증시간)
        3. test_score(최종점수) : 검증폴드의 점수
    - 최종결과는 '평균(mean)'으로 할 것

- 주의
    - 훈련세트를 섞어서 폴드를 나누지 않음
        - tarin_test_split()함수로 전체 데이터를 섞은 후 훈련세트 준비
        - 만약 교차검증을 할때 섞을 경우 분할기(splitter)를 지정
    - 폴드를 어떻게 나눌 것인지 기준설정
    - 기본적으로 회귀모델인 KFold 분할기를 사용
    - 분류 모델인 경우,
        target 클래스를 골고루 나누기 위해 StratifiedKFold 이용

<img src = "https://velog.velcdn.com/images%2Fheidi%2Fpost%2Fd9921883-ad1d-4855-b962-d4f162d91507%2Fimage.png" width = 450px>

In [5]:
from sklearn.model_selection import cross_validate

# 평가할 모델 객체를 첫번째 매개변수로 전달(직접 검증세트를 구분하지 않고 훈련세트 전체를 전달)
# dt : 결정트리 알고리즘 객체
scores = cross_validate(dt, train_input, train_target, cv = 5)
# cv : 폴드의 수(기본값 : 5)_반복횟수

print(scores)

"""
{'fit_time': array([0.00916362, 0.00838637, 0.00910163, 0.00918841, 0.00849676]),
'score_time': array([0.00124025, 0.00130463, 0.00137353, 0.00137806, 0.00139999]),
'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
"""

{'fit_time': array([0.02080321, 0.00971937, 0.01118636, 0.01313829, 0.00964594]), 'score_time': array([0.00192261, 0.00193024, 0.00206494, 0.00211382, 0.00195003]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


"\n{'fit_time': array([0.00916362, 0.00838637, 0.00910163, 0.00918841, 0.00849676]),\n'score_time': array([0.00124025, 0.00130463, 0.00137353, 0.00137806, 0.00139999]),\n'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}\n"

In [6]:
import numpy as np

print("성능평가(평균) : ",np.mean(scores['test_score']))
print("test           : ", dt.score(test_input, test_target))
# 결론 : train(0.855) < test(0.857) : 과소적합

성능평가(평균) :  0.855300214703487
test           :  0.8569230769230769


## StratifiedKFold
- cross_validate()는 섞어서 폴드를 준비하지 않기에 교차검증시 데이터 분할기
- train_test_split()로 세트분할 후 바로 셔플 및 분할을 진행할때 사용
    - n_splits : 폴드수 지정

In [7]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print("성능평가(평균) : ",np.mean(scores['test_score']))
print("test           : ", dt.score(test_input, test_target))
# 결론 : train(0.855) < test(0.857) : 과소적합

성능평가(평균) :  0.855300214703487
test           :  0.8569230769230769


In [8]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)#10-폴드 교차검증수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print("성능평가(평균) : ",np.mean(scores['test_score']))
print("test           : ", dt.score(test_input, test_target))
# 결론 : train(0.8574) > test(0.8569) : 적당

성능평가(평균) :  0.8574181117533719
test           :  0.8569230769230769


# 하이퍼파라미터 튜닝

- 결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아봐야함
- 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 골라야함
- 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
- 그 다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 변경 1 ~ 2, 5 ~ 6개의 매개변수를 제공
<br/><br/>
- 종류
    1. 모델파라미터 : 머신런닝 모델이 학습하는 파라미터
    2. 하이퍼파라미터 : 사용자가 지정해야만 하는 파라미터
<br/><br/>
- AutoML : 사람의 개입없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술

- 실행
    - 기존 : Max_depth를 최적으로 고정하고 min_sample_split을 바꿔가며 최적의 값을 찾는다.면 값이 함께 변경됨
        
        => 두개의 매개변수를 동시에 바꿔가며 최적의 값을 찾아야함 -> 다중for문(복잡함)
    - 개선 :  GridSearchCV : sklearn에서 제공하는 그리드 서치(Grid Search)가 제공,cv(cross_validate)

## 그리드서치(GridSearchCV)
- param
    - critertion : 분할 품질을 측정하는 기능(default : gini)
    - splitter : 각 노드에서 분할을 선택하는 데 사용되는 전략
    - max_depth : 트리의 최대 깊이(값이 클수록 모델의 복잡도가 올라감)
    - min_samples_split : 자식 노드를 분할하는데 필요한 최소 샘플수(default : 2)
        - 클수록 가지치기(과대적합 방지), 작을수록 정확하기(과대적합)
    - min_samples_leaf : 리프 노드에 있어야 할 최소 샘플수(default : 1)
    - min_weight_fraction_leaf : min_samples_leaf와 같지만 가중치가 부여된 샘플 수에서의 비율
    - max_features : 각 노드에서 분할에 사용할 특징의 최대수
    - random_state : 난수 seed 설정
    - max_leaf_nodes : 리프 노드의 최대수
    - min_impurity_decrease : 최소 불손도
    - min_impurity_split : 나무 성장을 멈추기 위한 임계치
    - class_weight : 클래스 가중치
    - presort : 데이터 정렬 필요 여부
    - n_jobs : 병렬로 사용할 CPU의 수 지정(-1 : 모든 코어사용)
    - cv : 반복횟수 지정(default : 5)
        - 횟수가 너무 크면 cpu를 많이 잡아먹어서 시간이 오래걸림
- 과정
    1. 탐색할 매개변수 지정
    2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음
        => 그리드 서치에 저장
    3. 그리드 서치는 최상의 메개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련
        => 그리드 서치에 저장
<br/><br/>
- 결론
    - 기존 : 최적의 파라미터를 찾으면 전체 훈련세트를 재생성 후 훈련
    - 그리드서치 : 검증 점수가 가장 높은 모델의 매개변수 조합으로 **자동**으로 모델 훈련재개

In [13]:
#그리드서치 크로스검증용
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} #0.0001 ~ 증가
# 매개변수와 탐색할 값을 리스트형태(중요!!!)로 딕셔너리로 만듬

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs=-1)

#min_impurity_decrease 값 총 5개 * cv 매개변수 기본값은 5 =  5*5=25번 수행(cpu를 많이 잡아먹음)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
# 이렇게 최적화되어 훈련된 모델은 GridSearchCV 클래스로 만들어진 객체의 best_estimator_ 속성에 저장되어 있다.
# 최적의 매개변수는 best_params_ 속성에 저장되어 있다.
print(f"그리드서치를 통해 찾은 최적의 모델의 정확도 : {dt.score(train_input, train_target)*100:.2f}%")

그리드서치를 통해 찾은 최적의 모델의 정확도 : 97.06%


In [17]:
# 각 매개변수에서 수행한 교차검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어 있다.
print(f"최적의 매개변수 : {gs.best_params_}")
# r결론 : 0.0001로 훈련한 모델이 가장 좋음

최적의 매개변수 : {'min_impurity_decrease': 0.0001}


In [18]:
# 5번의 교차검증으로 얻은 점수를 출력
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
# 눈으로 보는 것보다 numpy를 이용해 argmax()함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
# 그 다음 이 인덱스를 사용해서 params 키에 저장된 매개변수를 출력해보자
# 이값이 최상의 검증 점술로 만든 매개변수의 조합이 된다.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


## 조합확인(best_params_) & 점수확인(cv_results_)
- 최상의 매개변수 조합 및 조합에 대한 검증점수확인

In [22]:
# 노드를 분할하기 위한 불순도 감소 최소량 지정 max_depth(트리의 깊이)
# min_samples_split 노드를 나누기 위한 최소 샘플 수
# numpy arange()함수는 첫번째 매개변수 값에서 시작해서 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듬

# 0.0001 ~ 0.001까지 0.0001을 더한 배열(두번째 매개변수는 포함되지 않음_arange특징) = 9번
# 파이썬 arange() 함수는 정수만 사용가능 5~20까지 1씩 증가 = 15개
# 교차 검증 회수 9*15*10 = 1350개 * 5-폴드교차검증 = 6750

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split':range(2,100,10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
# 최상의 매개변수 조합확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
# 최상의 교차점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


## 개선점
- 탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했는데 근거 부족(좁히거나 넓힐필요?)

- 매개변수의 값이 수치일때 값의 범위나 간격을 미리 정하기 어려울 수 있음
- 너무 많은 매개 변수 조건이 있어 그리드 서치 수행시간이 오래 걸림
- 해결 : 랜덤 서치를 사용

### 랜덤서치
-  매개변수의 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포도 객체를 전달
- 모두 주어진 범위에서 고르게 값을 뽑는다.(균등 분포에서 샘플링)
    - randint() : 정수값
    - uniform() : 실수값
    - .rvs(n) : n개 추출
<br/>

싸이파이 :  파이썬의 핵심 과학 라이브러리로 적분, 보간, 성형대수, 확률 등 포함한 수치 계산용 전용 라이브러리(코랩에 내장)

In [26]:
from scipy.stats import uniform, randint

rgen = randint(0,10) #10개의 숫자를 샘플링
rgen.rvs(10)
# 10개밖에 되지 않기 때문에 고르게 샘플링되는 것 같지 않지만, 샘플링 숫자를 늘리면 쉽게 확인 할수 있다.
"""
array([1, 7, 8, 1, 0, 1, 1, 9, 2, 6])
"""

array([1, 7, 8, 1, 0, 1, 1, 9, 2, 6])

In [28]:
import numpy as np

np.unique(rgen.rvs(1000), return_counts=True) # 1000개를 샘플링해서 각 숫자의 개수를 세어봄
"""
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 105, 107, 119,  89, 105, 100,  96,  98,  76]))
"""

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 105, 107, 119,  89, 105, 100,  96,  98,  76]))

In [30]:
#실수 : uniform()
ugen = uniform(0,1)
ugen.rvs(10) #난무발생기(랜덤과 유사)

array([0.70152549, 0.23321542, 0.9462409 , 0.86263702, 0.56159964,
       0.13055658, 0.11840679, 0.94322529, 0.39927253, 0.5248206 ])

## 재훈련(랜덤서치이용)

In [37]:
#탐색할 매개변수의 딕셔너리를 만들어봄
#min_samples_leaf를 탐색 대상에 추가
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 50),
          'min_samples_leaf' : randint(1,25)
          }
# 리프노드가 되기 위한 최소 샘플 개수(자신노드의 샘플수가 이 값보다 작으면 분)

In [38]:
# 샘플링 회수는 sklearn의 램덤 처시 클래시인 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                        params, n_iter=100, n_jobs = -1, random_state=42)
#n_iter = 100 : 총 100번의 샘플링하여 교차 검증을 수행(최적의 매개변수 조합을 찾음)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79af2496b520>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79af2496ab30>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79af2496a650>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79af24897a90>},
                   random_state=42)

In [39]:
#최상의 조합 확인
print(gs.best_params_)
# 39 / 0.00034102546602601173 / 7 / 13

{'max_depth': 28, 'min_impurity_decrease': 0.000404781258158029, 'min_samples_leaf': 6, 'min_samples_split': 9}


In [40]:
# 최상의 교차점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8693486710594508


## 훈련평가

In [41]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))
#결론 : 검증(86.95%)> test(86.00%) : 적당

0.8576923076923076
